## Finch backend for `sparse`

<a href="https://colab.research.google.com/github/pydata/sparse/blob/main/examples/sparse_finch.ipynb" target="_blank">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" />
</a> to download and run.

In [1]:
!pip install 'sparse[finch]==0.16.0a9' scipy
!export SPARSE_BACKEND=Finch

# let's make sure we're using Finch backend
import os

os.environ["SPARSE_BACKEND"] = "Finch"
CI_MODE = os.getenv("CI_MODE", default=False)

ERROR: Exception:
Traceback (most recent call last):
  File "/Users/dealeon/projects/sparse/sparse-dea/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/Users/dealeon/projects/sparse/sparse-dea/lib/python3.12/site-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/dealeon/projects/sparse/sparse-dea/lib/python3.12/site-packages/pip/_internal/commands/install.py", line 324, in run
    session = self.get_default_session(options)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/dealeon/projects/sparse/sparse-dea/lib/python3.12/site-packages/pip/_internal/cli/index_command.py", line 71, in get_default_session
    self._session = self.enter_context(self._build_session(options))
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/deale

In [ ]:
import importlib
import time

import sparse

import matplotlib.pyplot as plt

import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as splin

assert sparse.BackendType.Finch == sparse.BACKEND

In [ ]:
tns = sparse.asarray(np.zeros((10, 10)))  # offers a no-copy constructor for NumPy as scipy.sparse inputs

s1 = sparse.random((100, 10), density=0.01)  # creates random COO tensor
s2 = sparse.random((100, 100, 10), density=0.01)
s2 = sparse.asarray(s2, format="csf")  # can be used to rewrite tensor to a new format

result = sparse.tensordot(s1, s2, axes=([0, 1], [0, 2]))

total = sparse.sum(result * result)
print(total)

### Example: least squares - closed form

In [ ]:
y = sparse.random((100, 1), density=0.08)
X = sparse.random((100, 5), density=0.08)
X = sparse.asarray(X, format="csc")
X_lazy = sparse.lazy(X)

X_X = sparse.compute(sparse.permute_dims(X_lazy, (1, 0)) @ X_lazy, verbose=True)

X_X = sparse.asarray(X_X, format="csc")  # move back from dense to CSC format

inverted = splin.inv(X_X)  # dispatching to scipy.sparse.sparray

b_hat = (inverted @ sparse.permute_dims(X, (1, 0))) @ y

print(b_hat.todense())

## Benchmark plots

In [ ]:
ITERS = 3
rng = np.random.default_rng(0)

In [ ]:
plt.style.use("seaborn-v0_8")
plt.rcParams["figure.dpi"] = 400
plt.rcParams["figure.figsize"] = [8, 4]

In [ ]:
def benchmark(func, info, args) -> float:
    start = time.time()
    for _ in range(ITERS):
        func(*args)
    elapsed = time.time() - start
    return elapsed / ITERS

In [ ]:
print("MTTKRP Example:\n")

os.environ[sparse._ENV_VAR_NAME] = "Numba"
importlib.reload(sparse)

configs = [
    {"I_": 100, "J_": 25, "K_": 10, "L_": 10, "DENSITY": 0.001},
    {"I_": 100, "J_": 25, "K_": 100, "L_": 10, "DENSITY": 0.001},
    {"I_": 100, "J_": 25, "K_": 100, "L_": 100, "DENSITY": 0.001},
    {"I_": 1000, "J_": 25, "K_": 100, "L_": 100, "DENSITY": 0.001},
    {"I_": 1000, "J_": 25, "K_": 1000, "L_": 100, "DENSITY": 0.001},
    {"I_": 1000, "J_": 25, "K_": 1000, "L_": 1000, "DENSITY": 0.001},
]
nonzeros = [10000, 100_000, 1_000_000, 10_000_000, 100_000_000, 1_000_000_000]

if CI_MODE:
    configs = configs[:1]
    nonzeros = nonzeros[:1]

finch_times = []
numba_times = []

for config in configs:
    B_sps = sparse.random((config["I_"], config["K_"], config["L_"]), density=config["DENSITY"], random_state=rng) * 10
    D_sps = rng.random((config["L_"], config["J_"])) * 10
    C_sps = rng.random((config["K_"], config["J_"])) * 10

    # ======= Finch =======
    os.environ[sparse._ENV_VAR_NAME] = "Finch"
    importlib.reload(sparse)

    B = sparse.asarray(B_sps.todense(), format="csf")
    D = sparse.asarray(np.array(D_sps, order="F"))
    C = sparse.asarray(np.array(C_sps, order="F"))

    @sparse.compiled
    def mttkrp_finch(B, D, C):
        return sparse.sum(B[:, :, :, None] * D[None, None, :, :] * C[None, :, None, :], axis=(1, 2))

    # Compile
    result_finch = mttkrp_finch(B, D, C)
    # Benchmark
    time_finch = benchmark(mttkrp_finch, info="Finch", args=[B, D, C])

    # ======= Numba =======
    os.environ[sparse._ENV_VAR_NAME] = "Numba"
    importlib.reload(sparse)

    B = sparse.asarray(B_sps, format="gcxs")
    D = D_sps
    C = C_sps

    def mttkrp_numba(B, D, C):
        return sparse.sum(B[:, :, :, None] * D[None, None, :, :] * C[None, :, None, :], axis=(1, 2))

    # Compile
    result_numba = mttkrp_numba(B, D, C)
    # Benchmark
    time_numba = benchmark(mttkrp_numba, info="Numba", args=[B, D, C])

    np.testing.assert_allclose(result_finch.todense(), result_numba.todense())
    finch_times.append(time_finch)
    numba_times.append(time_numba)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)

ax.plot(nonzeros, finch_times, "o-", label="Finch")
ax.plot(nonzeros, numba_times, "o-", label="Numba")
ax.grid(True)
ax.set_xlabel("no. of elements")
ax.set_ylabel("time (sec)")
ax.set_title("MTTKRP")
ax.set_xscale("log")
ax.set_yscale("log")
ax.legend(loc="best", numpoints=1)

plt.show()

In [ ]:
print("SDDMM Example:\n")

configs = [
    {"LEN": 10, "DENSITY": 0.1},
    {"LEN": 50, "DENSITY": 0.05},
    {"LEN": 100, "DENSITY": 0.01},
    {"LEN": 500, "DENSITY": 0.005},
    {"LEN": 1000, "DENSITY": 0.001},
    {"LEN": 5000, "DENSITY": 0.00005},
    {"LEN": 10000, "DENSITY": 0.00001},
]
size_n = [10, 50, 100, 500, 1000, 5000, 10000]

if CI_MODE:
    configs = configs[:1]
    size_n = size_n[:1]

finch_times = []
numba_times = []
scipy_times = []

for config in configs:
    LEN = config["LEN"]
    DENSITY = config["DENSITY"]

    a_sps = rng.random((LEN, LEN)) * 10
    b_sps = rng.random((LEN, LEN)) * 10
    s_sps = sps.random(LEN, LEN, format="coo", density=DENSITY, random_state=rng) * 10
    s_sps.sum_duplicates()

    # ======= Finch =======
    os.environ[sparse._ENV_VAR_NAME] = "Finch"
    importlib.reload(sparse)

    s = sparse.asarray(s_sps)
    a = sparse.asarray(np.array(a_sps, order="F"))
    b = sparse.asarray(np.array(b_sps, order="C"))

    @sparse.compiled
    def sddmm_finch(s, a, b):
        return sparse.sum(
            s[:, :, None] * (a[:, None, :] * sparse.permute_dims(b, (1, 0))[None, :, :]),
            axis=-1,
        )

    # Compile
    result_finch = sddmm_finch(s, a, b)
    # Benchmark
    time_finch = benchmark(sddmm_finch, info="Finch", args=[s, a, b])

    # ======= Numba =======
    os.environ[sparse._ENV_VAR_NAME] = "Numba"
    importlib.reload(sparse)

    s = sparse.asarray(s_sps)
    a = a_sps
    b = b_sps

    def sddmm_numba(s, a, b):
        return s * (a @ b)

    # Compile
    result_numba = sddmm_numba(s, a, b)
    # Benchmark
    time_numba = benchmark(sddmm_numba, info="Numba", args=[s, a, b])

    # ======= SciPy =======
    def sddmm_scipy(s, a, b):
        return s.multiply(a @ b)

    s = s_sps.asformat("csr")
    a = a_sps
    b = b_sps

    result_scipy = sddmm_scipy(s, a, b)
    # Benchmark
    time_scipy = benchmark(sddmm_scipy, info="SciPy", args=[s, a, b])

    finch_times.append(time_finch)
    numba_times.append(time_numba)
    scipy_times.append(time_scipy)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)

ax.plot(size_n, finch_times, "o-", label="Finch")
ax.plot(size_n, numba_times, "o-", label="Numba")
ax.plot(size_n, scipy_times, "o-", label="SciPy")

ax.grid(True)
ax.set_xlabel("size N")
ax.set_ylabel("time (sec)")
ax.set_title("SDDMM")
ax.set_xscale("log")
# ax.set_yscale('log')
ax.legend(loc="best", numpoints=1)

plt.show()